# Twitter Automation Scanner - Semantic Content analysis

## Install required dependencies

- [langdetect](https://github.com/Mimino666/langdetect) - langugage detection library
- [matplotlib](https://matplotlib.org) - Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python. Matplotlib makes easy things easy and hard things possible.
- [wordcloud](https://github.com/amueller/word_cloud) - A little word cloud generator in Python
- [spaCy](https://spacy.io) - spaCy is a library for advanced Natural Language Processing. spaCy comes with pretrained pipelines and currently supports tokenization and training for 70+ languages. It features state-of-the-art speed and neural network models for tagging, parsing, named entity recognition, text classification and more, multi-task learning with pretrained transformers.
- [NLTK](https://www.nltk.org) - NLTK stands for Natural Language Toolkit. NLTK is a platform for building Python programs to work with human language data. 

In [ ]:
pip install langdetect matplotlib wordcloud spacy nltk

## Load source JSON file

In [2]:
import json
import os

folder_walk = os.walk('sourcedata/user')
first_file_in_folder = next(folder_walk)[2][0]

report_file = open(f'{ os.getcwd() }/sourcedata/user/{first_file_in_folder}')
user_data = json.load(report_file)

## Detect the user language

In [ ]:
from langdetect import detect_langs

separator = ' '
tweets = list()

for tweet in user_data['tweets']:
    tweets.append(tweet['text'])

merged_user_posts = separator.join(tweets)
detected_langs_result = detect_langs(merged_user_posts)

print( 'For the provided report used language probabilities are:' )
for result in detected_langs_result:
    probability = round(result.prob * 100, 4)
    print(f'Detected language is { result.lang } with probability { probability }%')



## Define stop words

In [4]:
# Extended https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt
stop_words = [
    'a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ', 'ani', 'az', 'aż', 'bardziej',
    'bardzo', 'beda', 'bedzie', 'bez', 'będą', 'bede', 'będę', 'będzie', 'bo', 'bowiem', 'by', 'byc', 'być', 'byl',
    'byla', 'byli', 'bylo', 'byly', 'był', 'była', 'było', 'były', 'bynajmniej', 'cala', 'cali', 'caly', 'cała',
    'cały', 'ci', 'cie', 'ciebie', 'cię', 'co', 'cokolwiek', 'cos', 'coś', 'czasami', 'czasem', 'czemu', 'czy',
    'czyli', 'czym', 'daleko', 'da', 'dla', 'dlaczego', 'dlatego', 'do', 'dobrze', 'dokad', 'dokąd', 'dosc', 'dość',
    'duzo', 'dużo', 'dwa', 'dwaj', 'dwie', 'dwoje', 'dzis', 'dzisiaj', 'dziś', 'gdy', 'gdyby', 'gdyz', 'gdyż',
    'gdzie', 'gdziekolwiek', 'gdzies', 'gdzieś', 'go', 'i', 'ich', 'ile', 'im', 'inna', 'inne', 'inny', 'innych',
    'itd', 'iz', 'iż', 'ja', 'jak', 'jaka', 'jakas', 'jakaś', 'jakby', 'jaki', 'jakichs', 'jakichś', 'jakie',
    'jakieś', 'jakis', 'jakies', 'jakiś', 'jakiz', 'jakiż', 'jakkolwiek', 'jako', 'jakos', 'jakoś', 'ją', 'je',
    'jeden', 'jedna', 'jednak', 'jednakze', 'jednakże', 'jedno', 'jego', 'jej', 'jemu', 'jesli', 'jest', 'jestem',
    'jeszcze', 'jeśli', 'jezeli', 'jeżeli', 'juz', 'już', 'kazdy', 'każdy', 'kiedy', 'kilka', 'kims', 'kimś', 'kto',
    'ktokolwiek', 'ktora', 'ktore,', 'ktorego', 'ktorej', 'ktory', 'ktorych', 'ktorym', 'ktorzy', 'ktos', 'ktoś',
    'która', 'które', 'którego', 'której', 'który', 'których', 'którym', 'którzy', 'ku', 'lat', 'lecz', 'lub',
    'ma', 'mają', 'maja', 'mało', 'mam', 'mi', 'miedzy', 'między', 'mimo', 'mna', 'mną', 'mnie', 'moga', 'mogą',
    'moi', 'moim', 'moj', 'moja', 'moje', 'moze', 'mozliwe', 'mozna', 'może', 'możliwe', 'mój', 'można', 'mu',
    'musi', 'my', 'na', 'nad', 'nam', 'nami', 'nas', 'nasi', 'nasz', 'nasza', 'nasze', 'naszego', 'naszych',
    'natomiast', 'natychmiast', 'nawet', 'nia', 'niby', 'nic', 'nią', 'nie', 'nich', 'niech', 'niego', 'niej',
    'niemu', 'niemu', 'nigdy', 'nim', 'nimi', 'niz', 'niż', 'no', 'np', 'nr', 'o', 'obok', 'oby', 'od', 'ok',
    'około', 'on', 'ona', 'one', 'oni', 'ono', 'oraz', 'oto', 'owszem', 'pan', 'pana', 'pani', 'po', 'pod',
    'podczas', 'pomimo', 'ponad', 'poniewaz', 'ponieważ', 'powinien', 'powinna', 'powinni', 'powinno', 'poza',
    'prawie', 'przeciez', 'przecież', 'przed', 'przede', 'przedtem', 'przez', 'przy', 'roku', 'rowniez',
    'również', 'rt', 'sam', 'sama', 'są', 'sie', 'się', 'skad', 'skąd', 'soba', 'sobą', 'sobie', 'sposob',
    'sposób', 'swoje', 't', 'tt', 'ta', 'tak', 'taka', 'taki', 'takie', 'takze', 'także', 'tam', 'te', 'tę', 
    'tego', 'tej', 'ten', 'teraz', 'też', 'to', 'toba', 'tobą', 'tobie', 'totez', 'toteż', 'totobą', 'trzeba',
    'tu', 'tutaj', 'twoi', 'twoim', 'twoj', 'twoja', 'twoje', 'twój', 'twym', 'ty', 'tych', 'tylko', 'tym',
    'u', 'w', 'wam', 'wami', 'was', 'wasz', 'wasza', 'wasze', 'we', 'według', 'wiele', 'wielu', 'więc', 'więcej',
    'wlasnie', 'właśnie', 'ws', 'wszyscy', 'wszystkich', 'wszystkie', 'wszystkim', 'wszystko', 'wtedy', 'wy', 'z', 'za',
    'zaden', 'zadna', 'zadne', 'zadnych', 'zapewne', 'zawsze', 'ze', 'zeby', 'zeznowu', 'zł', 'znow', 'znowu',
    'znów', 'zostal', 'został', 'żaden', 'żadna', 'żadne', 'żadnych', 'że', 'żeby', ' ', 'itp', 'cc', 'ów', 'mieć'
    'miał', 'miała', 'mieli', 'tj', 'xd', 'owe', 'temu', 'tych', 'zaś', 'zatem'
] 

## Prepare a word cloud

In [ ]:
import re

import matplotlib.pyplot as plt
from wordcloud import WordCloud

def black_color_func(word, font_size, position,orientation,random_state=None, **kwargs):
    return("hsl(0,100%, 1%)")

merged_tweets = separator.join(tweets).lower()

merged_tweets = re.sub(r'http\S+', '', merged_tweets)

words = merged_tweets.split()

trimmed_words = list()

for word in words:
    word = re.sub('\W+','', word)
    if word not in stop_words and not word.startswith('@'):
        trimmed_words.append( word )

text = ' '.join(trimmed_words)

wordcloud = WordCloud(background_color="white", width=3000, height=2000).generate(text)

wordcloud.recolor(color_func = black_color_func)

plt.figure(figsize=[10, 15])

plt.imshow(wordcloud)

plt.axis("off")

## Named Entity Recognition

In [6]:
import spacy
from spacy import displacy
from collections import Counter

nlp = spacy.load("pl_core_news_lg")

content = nlp(merged_tweets)

In [ ]:
# render text with marked named entities
displacy.render(content, jupyter=True, style='ent')

## Count lemmas

In [ ]:
import nltk
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures

lemmas = list()

content = nlp(text)

for token in content:
  lemmas.append(token.lemma_)

print('Lemmas count for user is:', len(Counter(lemmas).items()))
print('Analyzed tweets count: ', len(tweets))
print('Total count of words (excl. stop words):', len(trimmed_words))

## Discover collocations

In [ ]:
finder = BigramCollocationFinder.from_words(lemmas)
bgm = BigramAssocMeasures()
collocations = {bigram: pmi for bigram, pmi in finder.score_ngrams(bgm.mi_like)}

collocations